In [1]:
# Importação das bibliotecas necessárias
import cv2
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as spla
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow # Para exibir imagens

In [1]:
# Funções Principais

def montar_sistema_poisson(img, mascara):
  """
  Monta o sistema linear esparso Ax=b baseado na equação de Poisson.
  A equação é: 4*u(i,j) - u(i-1,j) - u(i+1,j) - u(i,j-1) - u(i,j+1) = b_val
  """

  altura, largura = img.shape

  # Identificando incógnitas (pixels ausentes/danificados)
  # np.where(mascara) retorna uma tupla (array_linhas, array_colunas)
  pixels_ausentes = np.where(mascara)

  # N é o número total de pixels
  N = len(pixels_ausentes[0])

  # Mapear coordenadas (i, j) para o índice k (0 a N-1) do sistema linear
  mapa_pixels = np.full(img.shape, -1, dtype=int)
  mapa_pixels[pixels_ausentes] = np.arange(N)

  # Montar a matriz A (N x N) e vetor b (N x 1)
  A = sp.lil_matrix((N, N))
  b = np.zeros(N)

  for k in range(N):
        i, j = pixels_ausentes[0][k], pixels_ausentes[1][k]

        # Define a diagonal principal
        A[k, k] = 4

        # Analisa os 4 vizinhos
        for di, dj in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            viz_i, viz_j = i + di, j + dj

            if mascara[viz_i, viz_j]:
                viz_k = mapa_pixels[viz_i, viz_j] # Encontra o índice 'm' do vizinho
                A[k, viz_k] = -1 # Define A[k, m] = -1

            else:
                b[k] += img[viz_i, viz_j]

  # Converte A para o formato CSR (Compressed Sparse Row) para um cálculo rápido
  A_csr = A.tocsr()

  return A_csr, b, mapa_pixels


def resolver_sistema_lu(A, b):
    """
    Resolve o sistema Ax=b usando a decomposição LU esparsa.
    """

    # splu cria um objeto que contém a decomposição LU da matriz A
    # Isso é muito mais rápido do que inverter A
    print("Calculando Decomposição LU...")
    # .tocsc() é o formato ideal para a fatoração LU
    lu_obj = spla.splu(A.tocsc())

    print("Resolvendo para x (Ly=b e Ux=y)...")
    # lu_obj.solve(b) resolve o sistema completo usando a fatoração
    x = lu_obj.solve(b)

    print("Sistema resolvido.")
    return x

def reconstruir_imagem(img, mascara, x):
  """
  Cria a imagem final preenchendo os pixels ausentes com a solução x
  """
  # Cria uma cópia da imagem original para não alterá-la
  img_restaurada = img.copy()

  # Pega as coordenadas dos pixels ausentes
  pixels_ausentes = np.where(mascara)

  # Preenche os locais da máscara com os valores calculados de x
  img_restaurada[pixels_ausentes] = x

  # Garante que os valores permaneçam no intervalo válido [0, 1]
  img_restaurada = np.clip(img_restaurada, 0, 1)

  return img_restaurada

  def processar_canal(canal_img, canal_mascara):
    """Orquestra o processo todo para um único canal (tons de cinza)"""

    print("Passo A: Montando sistema Poisson...")
    A, b, mapa_pixels = montar_sistema_poisson(canal_img, canal_mascara)

    print("Passo B: Resolvendo com Decomposição LU...")
    x = resolver_sistema_lu(A, b)

    print("Passo C: Reconstruindo canal...")
    canal_restaurado = reconstruir_imagem(canal_img, canal_mascara, x)

    return canal_restaurado

In [2]:
# --- GERA IMAGEM DE TESTE (Original) ---
img_teste = np.zeros((100, 100), dtype=float)
cv2.circle(img_teste, (50, 50), 25, 1.0, 2) # Desenha um círculo branco

# --- GERA MÁSCARA DE TESTE (Danificada) ---
# Máscara é Falsa (0) onde conhecemos, True (1) onde não conhecemos
mascara_teste = np.zeros((100, 100), dtype=bool)
# Define uma faixa retangular no meio como "ausente" (True)
mascara_teste[40:60, :] = True

# --- CRIA A IMAGEM DANIFICADA (para visualização) ---
img_danificada = img_teste.copy()
img_danificada[mascara_teste] = 0.5 # Pinta a área danificada de cinza

# --- EXECUTA A RESTAURAÇÃO ---
print("--- INICIANDO TESTE EM TONS DE CINZA ---")
img_restaurada_teste = processar_canal(img_teste, mascara_teste)
print("--- TESTE EM TONS DE CINZA CONCLUÍDO ---")

# --- EXIBE OS RESULTADOS ---
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.title("Original (Teste)")
plt.imshow(img_teste, cmap='gray')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.title("Danificada (Máscara)")
plt.imshow(img_danificada, cmap='gray')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.title("Restaurada com LU")
plt.imshow(img_restaurada_teste, cmap='gray')
plt.axis('off')

plt.tight_layout()
plt.show()

NameError: name 'np' is not defined